In [3]:
import trueskill
import pandas as pd 


In [4]:
sp_stats_df = pd.read_csv('csv_files/sp_stats.csv')
sp_stats_df.head()

,Name,Team,W,L,SV,G,GS,IP,K/9,BB/9,HR/9,BABIP,LOB%,GB%,HR/FB,ERA,FIP,xFIP,WAR,playerid
0,Gerrit Cole,Astros,20,5,0,33,33,212.1,13.82,2.03,1.23,0.275,83.3%,40.3%,16.9%,2.50,2.64,2.48,7.4,13125
1,Jacob deGrom,Mets,11,8,0,32,32,204.0,11.25,1.94,0.84,0.282,81.8%,44.4%,11.2%,2.43,2.67,3.11,7.0,10954
2,Lance Lynn,Rangers,16,11,0,33,33,208.1,10.63,2.55,0.91,0.322,74.4%,40.3%,9.9%,3.67,3.13,3.85,6.8,2520
3,Max Scherzer,Nationals,11,7,0,27,27,172.1,12.69,1.72,0.94,0.321,78.7%,40.8%,11.6%,2.92,2.45,2.88,6.5,3137
4,Justin Verlander,Astros,21,6,0,34,34,223.0,12.11,1.70,1.45,0.218,88.4%,35.9%,16.0%,2.58,3.27,3.18,6.4,8700


In [5]:
sp_df = sp_stats_df[['Name', 'WAR'
]]

In [35]:
sp_df.head()

,Name,WAR
0,Gerrit Cole,7.4
1,Jacob deGrom,7.0
2,Lance Lynn,6.8
3,Max Scherzer,6.5
4,Justin Verlander,6.4


In [42]:
import math
import itertools
sp1 = 'Cole'
sp2 = 'deGrom'
def win_probability(sp1, sp2):
    delta_mu = sum(r.mu for r in sp1) - sum(r.mu for r in sp2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(sp1, sp2))
    size = len(sp1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = trueskill.global_env()
    return ts.cdf(delta_mu / denom)

In [43]:
win_probability(sp1, sp2)

AttributeError: 'str' object has no attribute 'mu'

In [ ]:
def add_trueskill_ratings(df):
    '''
    adds columns with trueskill ratings to df
    https://www.microsoft.com/en-us/research/project/trueskill-ranking-system/
    '''
    ratings = {}
    for x in df.home_team_abbr.unique():
        ratings[x]=25
    for x in df.away_team_abbr.unique():
        ratings[x]=25

    home_trueskill_pre = []
    away_trueskill_pre = []
    for i, r in df.iterrows():
        # get pre-match trueskill ratings from dict
        home_trueskill_pre.append(ratings[r.home_team_abbr])
        away_trueskill_pre.append(ratings[r.away_team_abbr])

        if r.date < df.date.max():
            #doubleheaders get screwed up if we do this on current day
            # update ratings dictionary with post-match ratings
            ts1 = Rating(ratings[r.home_team_abbr])
            ts2 = Rating(ratings[r.away_team_abbr])
            if r.home_team_win==1:
                ts1, ts2 = rate_1vs1(ts1, ts2)
            else:
                ts2, ts1 = rate_1vs1(ts2, ts1)
            ratings[r.home_team_abbr] = ts1.mu
            ratings[r.away_team_abbr] = ts2.mu

    df['home_trueskill_pre']= home_trueskill_pre
    df['away_trueskill_pre']= away_trueskill_pre
    df['ts_diff'] = df.home_trueskill_pre-df.away_trueskill_pre
    
    df.replace({np.inf: 0}, inplace=True)
    return df

KeyError: 1